In [1]:
import quandl
import yfinance as yf
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
from pandas import Timedelta, to_datetime
from datetime import datetime, date

pd.set_option('display.max_rows', 110)



# Black-Scholes

In [2]:
#import numpy as np
import scipy.stats as stats


class BlackScholes(object):
    """
        Attributes:
        S: float - Spot
        K: float - Strike
        tao : float - Time to Maturity
        r: float - Risk-Free Rate
        sigma : float - Volatility
        div : float - dividend rate
        d1: float
        d2: float
    """

    def __init__(self, S, K, tao, r, sigma, div=0):

        self.S = S
        self.K = K
        self.tao = tao
        self.r = r
        self.sigma = sigma
        self.div = div
        self.__compute_d1d2()

    def __compute_d1d2(self):

        self.d1 = (np.log(self.S / self.K) + (self.r - self.div + 0.5 * self.sigma ** 2) * self.tao) / (
            self.sigma * np.sqrt(self.tao))
        self.d2 = self.d1 - self.sigma * np.sqrt(self.tao)

    def get_data(self):
        return [self.get_Option_Price(), self.K, self.get_delta(), self.get_vega(), self.get_theta(), self.get_rho(), self.get_gamma()]


class European_Call(BlackScholes):
    """
        Attributes:
        S: float - Spot
        K: float - Strike
        tao : float - Time to Maturity
        r: float - Risk-Free Rate
        sigma : float - Volatility
        div : float - dividend rate
        d1: float
        d2: float
    """

    def __init__(self, S, K, tao, r, sigma, div=0):
        super().__init__(S, K, tao, r, sigma, div)

    def get_Option_Price(self):
        call_price = (self.S * np.exp(-self.div * self.tao) * stats.norm.cdf(self.d1, 0, 1) - self.K * np.exp(
            -self.r * self.tao) * stats.norm.cdf(self.d2, 0, 1))
        return call_price

    def get_delta(self):
        delta = np.exp(-self.div * self.tao) * stats.norm.cdf(self.d1, 0, 1)
        return delta

    def get_gamma(self):
        gamma = np.exp(-self.div * self.tao) * stats.norm.pdf(self.d1) / \
            (self.S * self.sigma * np.sqrt(self.tao))
        return gamma

    def get_theta(self):
        theta = -np.exp(-self.div * self.tao) * self.S * stats.norm.pdf(self.d1) * self.sigma / (
            2 * np.sqrt(self.tao)) - self.r * self.K * np.exp(-self.r * self.tao) * stats.norm.cdf(
            self.d2) + self.div * self.S * np.exp(
            -self.div * self.tao) * stats.norm.cdf(self.d1)
        return theta / 252

    def get_rho(self):
        rho = self.K * self.tao * \
            np.exp(-self.r * self.tao) * stats.norm.cdf(self.d2)
        return rho * 0.01

    def get_vega(self):
        vega = self.S * np.exp(-self.div * self.tao) * \
            stats.norm.pdf(self.d1) * np.sqrt(self.tao)
        return vega * 0.01


class European_Put(BlackScholes):
    """
        Attributes:
        S: float - Spot
        K: float - Strike
        tao : float - Time to Maturity
        r: float - Risk-Free Rate
        sigma : float - Volatility
        div : float - dividend Rate
        d1: float
        d2: float
    """

    def __init__(self, S, K, tao, r, sigma, div=0):
        super().__init__(S, K, tao, r, sigma, div)

    def get_Option_Price(self):
        put_price = (self.K * np.exp(-self.r * self.tao) * stats.norm.cdf(-self.d2, 0, 1) - self.S * np.exp(
                     -self.div * self.tao) * stats.norm.cdf(-self.d1, 0, 1))
        return put_price

    def get_delta(self):
        delta = -np.exp(-self.div * self.tao) * stats.norm.cdf(-self.d1)
        return delta

    def get_gamma(self):
        gamma = np.exp(-self.div * self.tao) * stats.norm.pdf(self.d1) / \
            (self.S * self.sigma * np.sqrt(self.tao))
        return gamma

    def get_theta(self):
        theta = -np.exp(-self.div * self.tao) * self.S * stats.norm.pdf(-self.d1) * self.sigma / (
            2 * np.sqrt(self.tao)) + self.r * self.K * np.exp(-self.r * self.tao) * stats.norm.cdf(
            -self.d2) - self.div * self.S * np.exp(
            -self.div * self.tao) * stats.norm.cdf(-self.d1)
        return theta / 252

    def get_rho(self):
        rho = -self.K * self.tao * \
            np.exp(-self.r * self.tao) * stats.norm.cdf(-self.d2, 0, 1)
        return rho * 0.01

    def get_vega(self):
        vega = self.S * np.exp(-self.div * self.tao) * \
            stats.norm.pdf(self.d1) * np.sqrt(self.tao)
        return vega * 0.01

# Risk Free Rate

In [3]:
class RiskFreeRate():
    def __init__(self, BANK_YEAR=360, BANK_MONTH=30):
        self.tbill = quandl.get("USTREASURY/YIELD")

        self.BANK_YEAR = BANK_YEAR
        self.BANK_MONTH = BANK_MONTH

        self.VALID_M = [1, 2, 3, 6]
        self.VALID_Y = [1, 2, 3, 5, 7, 10, 20, 30]
        self.dp = {}

    def _fuzzy(self, t, mode):
        if mode == 'YR':
            lst = self.VALID_Y
        else:
            lst = self.VALID_M

        min_diff = abs(t - lst[0])
        for i in range(1, len(lst)):
            if abs(t - lst[i]) < min_diff:
                min_diff = abs(t - lst[i])

        return lst[i]

    def r(self, d, tao):
        if tao in self.dp:
            return self.tbill.loc[d, self.dp[tao]] / 100

        month = round(tao * self.BANK_YEAR / self.BANK_MONTH)
        if month >= 12:
            year = round(month / 12)
            t = str(self._fuzzy(year, 'YR')) + ' YR'
        else:
            t = str(self._fuzzy(month, 'MO')) + ' MO'

        self.dp[tao] = t

        try:
            return self.tbill.loc[d, t] / 100
        except KeyError:
            d = to_datetime(d)
            while d not in self.tbill.index:
                d = d - Timedelta(days=1)
                return self.tbill.loc[d, t] / 100


if __name__ == '__main__':
    rf = RiskFreeRate()
    print(rf.tbill.index)
    r = rf.r('2020-11-11 00:00:00', 1)
    print(r)

DatetimeIndex(['1990-01-02', '1990-01-03', '1990-01-04', '1990-01-05',
               '1990-01-08', '1990-01-09', '1990-01-10', '1990-01-11',
               '1990-01-12', '1990-01-16',
               ...
               '2021-11-23', '2021-11-24', '2021-11-26', '2021-11-29',
               '2021-11-30', '2021-12-01', '2021-12-02', '2021-12-03',
               '2021-12-06', '2021-12-07'],
              dtype='datetime64[ns]', name='Date', length=7991, freq=None)
0.0175


# MonteCarlo

In [4]:

from typing import Tuple
import numpy as np
from numpy.polynomial import Polynomial
from numpy.polynomial.polynomial import polyval
from scipy import stats


class MonteCarloBase():
    def __init__(self, n, steps, alpha=0.05, epsilon=None):
        """
            Attributes:
            S: float - Spot Price
            K: float - Strike Price
            tao : float - Time to Maturity
            r: float - risk-free rate(TBD)
        """
        self.n = n
        self.steps = steps
        self.alpha = alpha
        self.epsilon = epsilon

    def gen_epsilon(self):
        return np.random.normal(0, 1, (self.n, self.steps + 1))

    def _err(self):
        Z = stats.norm.ppf(1 - self.alpha/2)
        self.err = Z * np.std(self.path[:, -1]) / np.sqrt(self.n)


class MonteCarloEuropean(MonteCarloBase):
    def __init__(self, n, steps, S, K, sigma, tao, r, alpha=0.05, epsilon=None, path=None):
        """
            Attributes:
            S: float - Spot Price
            K: float - Strike Price
            tao : float - Time to Maturity
            r: float - risk-free rate(TBD)
        """
        super().__init__(n, steps, alpha, epsilon)

        self.S = S
        self.K = K
        self.sigma = sigma
        self.tao = tao
        self.r = r

        self.call = None
        self.put = None

        self.path = path
        if self.path is None:
            self._brownian_motion()

        self._price()
        self._greek()
        super()._err()

    def gen_path(self):
        dt = self.tao / self.steps
        # simulated stock prices at maturity
        path = self.S * np.exp(np.cumsum((self.r - 0.5 * np.power(self.sigma, 2))
                                         * dt + self.sigma * np.sqrt(dt) * self.epsilon, axis=1))
        return path

    def _brownian_motion(self, new_epsilon=False, new_path=False):
        # random variables
        if self.epsilon is None or new_epsilon:
            self.epsilon = super().gen_epsilon()
        if self.path is None or new_epsilon or new_path:
            # small time step
            self.path = self.gen_path()

    def _price(self):
        expected_call = np.sum(np.maximum(
            self.path[:, -1] - self.K, 0)) / self.n
        expected_put = np.sum(np.maximum(
            self.K - self.path[:, -1], 0)) / self.n
        self.call = np.exp(-self.r * self.tao) * expected_call
        self.put = np.exp(-self.r * self.tao) * expected_put

    def _rerun(self):
        prev_call, prev_put = self.call, self.put
        self._brownian_motion(new_path=True)
        self._price()
        new_call, new_put = self.call, self.put
        self.call, self.put = prev_call, prev_put
        return new_call, new_put

    def _diff(self) -> Tuple[float]:
        new_call, new_put = self._rerun()
        return new_call - self.call, new_put - self.put

    # 1st order greeks
    def _delta(self, dS):
        self.S += dS
        call_delta, put_delta = self._diff()
        self.S -= dS
        return call_delta / dS, put_delta / dS

    def _vega(self, dsigma):
        self.sigma += dsigma
        call_vega, put_vega = self._diff()
        self.sigma -= dsigma
        return call_vega / dsigma * 0.01, put_vega / dsigma * 0.01

    def _theta(self, dtao):
        self.tao += dtao
        call_theta, put_theta = self._diff()
        self.tao -= dtao
        return call_theta, put_theta

    def _rho(self, dr):
        self.r += dr
        call_rho, put_rho = self._diff()
        self.r -= dr
        return call_rho / dr * 0.01, put_rho / dr * 0.01

    # 2nd order greeks
    def _gamma(self, dS):
        self.S += dS
        call_left, put_left = self._rerun()
        self.S -= dS

        self.S -= dS
        call_right, put_right = self._rerun()
        self.S += dS

        call_gamma = call_left - 2 * self.call + call_right
        put_gamma = put_left - 2 * self.put + put_right
        return call_gamma / (dS ** 2), put_gamma / (dS ** 2)

    def _greek(self):
        if not (self.call and self.put):
            self._price()

        dS = 0.01
        self.call_delta, self.put_delta = self._delta(dS)

        dsigma = 0.0001
        self.call_vega, self.put_vega = self._vega(dsigma)

        dtao = -1 / 252
        self.call_theta, self.put_theta = self._theta(dtao)

        dr = 0.0001
        self.call_rho, self.put_rho = self._rho(dr)

        self.call_gamma, self.put_gamma = self._gamma(dS)

    def call_data(self):
        return [self.call, self.K, self.call_delta, self.call_vega, self.call_theta, self.call_rho, self.call_gamma]

    def put_data(self):
        return [self.put, self.K, self.put_delta, self.put_vega, self.put_theta, self.put_rho, self.put_gamma]


class MonteCarloAmerican(MonteCarloEuropean):
    def __init__(self, n, steps, S, K, sigma, tao, r, alpha=0.05, epsilon=None, path=None):
        super().__init__(n, steps, S, K, sigma, tao, r, alpha, epsilon, path)

    def _price(self):
        # The starting cash flow matrix, aka excercising at the last time step
        cf_call = np.ndarray(self.path.shape)
        cf_put = np.ndarray(self.path.shape)

        cf_call[:, -1] = self.path[:, -1] - self.K
        cf_put[:, -1] = self.K - self.path[:, -1]

        last_call = cf_call[:, -1]
        last_put = cf_put[:, -1]

        last_call[last_call < 0] = 0
        last_put[last_put < 0] = 0

        # col represent n simulated prices at time step t
        for col in range(self.path.shape[1] - 2, 0, -1):
            stocks = self.path[:, col]
            curr_call = cf_call[:, col]
            curr_put = cf_put[:, col]

            # We only want to consider early excercise if it is profitable
            call_mask = stocks > self.K
            put_mask = stocks < self.K

            # X: the current stock price
            X_call = stocks[call_mask]
            X_put = stocks[put_mask]

            # Y: the discounted cashflow at t, aka cf at (t + 1) * e^-r
            Y_call = last_call[call_mask] * np.exp(-self.r)
            Y_put = last_put[put_mask] * np.exp(-self.r)

            # Regress Y on X and X^2
            coef_call = Polynomial.fit(X_call, Y_call, 2).convert().coef
            coef_put = Polynomial.fit(X_put, Y_put, 2).convert().coef

            # Predict the expected value of continuation according to above regression
            cont_call = polyval(stocks, coef_call)
            cont_call[cont_call < 0] = 0
            cont_put = polyval(stocks, coef_put)
            cont_put[cont_put < 0] = 0

            # Calculate the value of early excercise
            ex_call = stocks - self.K
            ex_call[ex_call < 0] = 0
            ex_put = self.K - stocks
            ex_put[ex_put < 0] = 0

            # The current cash flow, aka max of excercise now vs expected value of continuing
            # If we don't early excercise, we have 0 cash flow.
            # print(curr_call[(curr_call == ex_call) & (curr_call > 0)])
            curr_call = np.maximum(ex_call, cont_call)
            curr_call[(curr_call == cont_call)] = 0
            curr_put = np.maximum(ex_put, cont_put)
            curr_put[(curr_put == cont_put)] = 0

            cf_call[:, col] = curr_call
            cf_put[:, col] = curr_put

            # We set the future cash flow to 0 if we early excercise
            ex_mask_call = (curr_call == ex_call) & (curr_call > 0)
            ex_mask_put = (curr_put == ex_put) & (curr_put > 0)
            last_call[ex_mask_call] = 0
            last_put[ex_mask_put] = 0

            # Move on to the t - 1
            last_call = cf_call[:, col]
            last_put = cf_put[:, col]

        optimal_ex_call = np.argmax(cf_call, axis=1)
        optimal_ex_put = np.argmax(cf_put, axis=1)

        expected_call = cf_call[np.arange(self.n), optimal_ex_call]
        call = np.sum(expected_call * np.exp(-self.r *
                                             optimal_ex_call / self.steps)) / self.n

        expected_put = cf_put[np.arange(self.n), optimal_ex_put]
        put = np.sum(expected_put * np.exp(-self.r *
                                           optimal_ex_put / self.steps)) / self.n

        self.call = call
        self.put = put


if __name__ == '__main__':
    sigma = 0.2

    r = 0.01

    # time (years) until maturity
    tao = 1

    # assume 0 dividend
    div = 0.0

    # Number of random walk in monte-carlo
    n = 10000

    # Number of time steps in monte-carlo
    steps = 252

    S = 305

    K_c = 315

    # import sys
    # np.set_printoptions(threshold=sys.maxsize)
    # mc_eu = MonteCarloEuropean(n, steps, S, K_c, sigma, tao, r)
    # mc_am = MonteCarloAmerican(n, steps, S, K_c, sigma, tao, r)
    # print(mc_eu.call)
    # print(mc_am.call)
    # print(mc_eu.put)
    # print(mc_am.put)
    # print(mc_eu.call_rho)
    # print(mc_am.call_rho)
    # print(mc.cf_call[:, -5])
    # print(np.argmax(mc.cf_call, axis=1))
    # print(mc.cf_call.shape)
    # no_repeat = True
    # for r in mc.cf_call[::-1]:
    #     print(r)
    #     if len(np.unique(r)) > 2:
    #         no_repeat = False
    #         break
    #     break
    # print(no_repeat)

# Delta Hedge

In [5]:
from numpy.core.defchararray import index

class HedgedPortfolio():
    def __init__(self, stock_price: pd.Series, K, tao, sigma, TRADING_YEAR=252) -> None:
        self.TRADING_YEAR = TRADING_YEAR

        self.stock_price = stock_price
        self.df = pd.DataFrame(
            columns=['time', 'stock_price', 'option_price', 'delta', 'pnl_unhedged', 'pnl'])
        self.rf = RiskFreeRate(TRADING_YEAR=self.TRADING_YEAR)
        self.sigma = sigma
        self.K = K
        self.tao = tao

    def pnl(self):
        r = self.rf.r(self.stock_price.index[0], self.tao)
        self.r = r
        for i, d in enumerate(self.stock_price.index):
            S = self.stock_price[d]
            t = (self.tao * self.TRADING_YEAR - i) / self.TRADING_YEAR

            call = European_Call(S, self.K, t, r, self.sigma)
            price = call.get_Option_Price()
            delta = call.get_delta()

            self.df.loc[i] = [i, S, price, delta, 0, 0]

            if i == 0:
                continue

            prev_price = self.df.loc[i - 1, 'option_price']
            prev_S = self.df.loc[i - 1, 'stock_price']
            prev_delta = self.df.loc[i - 1, 'delta']

            pnl_unhedged = (price - prev_price) * 100
            pnl = pnl_unhedged - prev_delta * round((S - prev_S)) * 100

            self.df.loc[i, 'pnl_unhedged'] = pnl_unhedged
            self.df.loc[i, 'pnl'] = pnl

# Notebook

In [6]:
%load_ext autoreload
%autoreload 2

from IPython.core.display import display, HTML
display(HTML("<style>div.output_scroll { height: 60em; }</style>"))
display(HTML("<style>.container { width:100% !important; }</style>"))

In [7]:
stock = 'MSFT'

start = '2020-10-13'
end = '2021-10-13'
data = yf.download([stock], start=start,
                     end=end, group_by='ticker').dropna()
close = data['Close'].to_numpy()

[*********************100%***********************]  1 of 1 completed


In [8]:
# Testing calculator on MSFT data

# volatility of stock price
historic = yf.download([stock], start='2019-10-13', end=start, group_by='ticker').dropna()
historic_close = historic['Close'].to_numpy()
sigma = np.sqrt(252)* np.std(np.log(historic_close[1:] / historic_close[:-1]))

# Using the spot Treasury rate closest to time of maturity
rf = RiskFreeRate()

# time (years) until maturity
tao = 1

# assume 0 dividend
div = 0.0

# Number of random walk in monte-carlo
n = 10000

# Number of time steps in monte-carlo
steps = 252

# Generate one path for all Monte Carlo model for consistancy
epsilon = MonteCarloBase(n, steps).gen_epsilon()


[*********************100%***********************]  1 of 1 completed


In [9]:
for d in data[0:1]['Close'].index:
    S = data.loc[d, 'Close']

    print('Stock Price S: ' + str(S))
    K_c = S + 10
    K_p = S - 10
    
    r = rf.r(d, tao)
    print('Risk Free Rate r: ' + str(r))

    print('Volatility sigma: ' + str(sigma))

    bs_call = European_Call(S, K_c, tao, r, sigma, div)
    bs_put = European_Put(S, K_p, tao, r, sigma, div)

    mc_eu_call = MonteCarloEuropean(n, steps, S, K_c, sigma, tao, r, epsilon=epsilon)
    mc_eu_put = MonteCarloEuropean(n, steps, S, K_p, sigma, tao, r, epsilon=epsilon)

    mc_am_call = MonteCarloAmerican(n, steps, S, K_c, sigma, tao, r, epsilon=epsilon)
    mc_am_put = MonteCarloAmerican(n, steps, S, K_c, sigma, tao, r, epsilon=epsilon)

    call_df = pd.DataFrame(columns=['Price', 'Strike', 'Delta', 'Vega', 'Theta', 'Rho', 'Gamma'], index = ['MC_EU','MC_AM', 'BSC'])
    call_df.loc['MC_EU'] = mc_eu_call.call_data()
    call_df.loc['MC_AM'] = mc_am_call.call_data()
    call_df.loc['BSC'] = bs_call.get_data()

    put_df = pd.DataFrame(columns=['Price', 'Strike', 'Delta', 'Vega', 'Theta', 'Rho', 'Gamma'], index = ['MC_EU', 'MC_AM', 'BSC'])
    put_df.loc['MC_EU'] = mc_eu_put.put_data()
    put_df.loc['MC_AM'] = mc_am_put.put_data()
    put_df.loc['BSC'] = bs_put.get_data()

    print('Call: ')
    print(call_df)
    print('Put: ')
    print(put_df)
    print('\n')
print('Monte Carlo 95% CI error: ', mc_eu_call.err)

Stock Price S: 222.86000061035156
Risk Free Rate r: 0.0152
Volatility sigma: 0.42864359345461733
Call: 
           Price      Strike        Delta       Vega      Theta         Rho  \
MC_EU  35.775066  232.860001       0.5651   0.897111  -0.081845    0.906674   
MC_AM  64.163515  232.860001  16070.90042  108.66586  160.69112  105.756086   
BSC    35.211643  232.860001     0.558584   0.879479  -0.080183    0.892745   

                Gamma  
MC_EU        0.003017  
MC_AM  1617605.941605  
BSC          0.004131  
Put: 
           Price      Strike     Delta      Vega     Theta       Rho  \
MC_EU  31.115688  212.860001 -0.355575  0.839045 -0.064764 -1.106682   
MC_AM  32.767698  232.860001  0.153907   0.63779 -0.045441 -4.485097   
BSC    30.460231  212.860001 -0.360589  0.834228 -0.064265  -1.10821   

            Gamma  
MC_EU         0.0  
MC_AM  161.655364  
BSC      0.003919  


Monte Carlo 95% CI error:  2.0275612229613547


In [10]:
stock = 'ARKQ'

start = '2020-10-13'
end = '2021-10-13'

arkq = yf.Ticker(stock)

arkq_stock = arkq.history(start=start, end=end).dropna()
arkq_close = arkq_stock['Close'].to_numpy()

# ('2021-11-19', '2021-12-17', '2022-01-21', '2022-03-18', '2022-06-17', '2023-01-20')
arkq_option = arkq.option_chain('2022-06-17')
arkq_call = arkq_option.calls
arkq_put = arkq_option.puts

In [11]:
# Historical volatility
arkq_historic = yf.download([stock], start='2019-10-13', end=start, group_by='ticker').dropna()
arkq_historic_close = arkq_historic['Close'].to_numpy()
vol_hist = np.sqrt(252)* np.std(np.log(arkq_historic_close[1:] / arkq_historic_close[:-1]))
print(vol_hist)

[*********************100%***********************]  1 of 1 completed
0.39438233474050616


In [12]:
# volatility of stock price
sigma = vol_hist

# Still using the spot Treasury rate closest to time of maturity
# rf

# time (years) until maturity
tao = 1

# assume 0 dividend
div = 0.0

# Number of random walk in monte-carlo
n = 10000

# Number of time steps in monte-carlo
steps = 252

In [13]:
r = rf.r(d, tao)
print('Risk Free Rate r: ' + str(r))

print('Volatility sigma: ' + str(sigma) + '\n')
for d in arkq_stock[0:1]['Close'].index:
    S = arkq_stock.loc[d, 'Close']

    print('Stock Price S: ' + str(S))
    K_c = S + 10
    K_p = S - 10

    bs_call = European_Call(S, K_c, tao, r, sigma, div)
    bs_put = European_Put(S, K_p, tao, r, sigma, div)

    mc_eu_call = MonteCarloEuropean(n, steps, S, K_c, sigma, tao, r, epsilon=epsilon)
    mc_eu_put = MonteCarloEuropean(n, steps, S, K_p, sigma, tao, r, epsilon=epsilon)

    call_df = pd.DataFrame(columns=['Price', 'Strike', 'Delta', 'Vega', 'Theta', 'Rho', 'Gamma'], index = ['MC_EU','MC_AM', 'BSC'])
    call_df.loc['MC_EU'] = mc_eu_call.call_data()
    call_df.loc['MC_AM'] = mc_am_call.call_data()
    call_df.loc['BSC'] = bs_call.get_data()

    put_df = pd.DataFrame(columns=['Price', 'Strike', 'Delta', 'Vega', 'Theta', 'Rho', 'Gamma'], index = ['MC_EU','MC_AM', 'BSC'])
    put_df.loc['MC_EU'] = mc_eu_put.put_data()
    put_df.loc['MC_AM'] = mc_am_put.put_data()
    put_df.loc['BSC'] = bs_put.get_data()

    print('Call: ')
    print(call_df)
    print('Put: ')
    print(put_df)
    print('\n')
print('Monte Carlo 95% CI error: ', mc_eu_call.err)

Risk Free Rate r: 0.0152
Volatility sigma: 0.39438233474050616

Stock Price S: 62.35628890991211
Call: 
           Price      Strike        Delta       Vega      Theta         Rho  \
MC_EU   6.696407   72.356289     0.449562    0.25123  -0.020969    0.214467   
MC_AM  64.163515  232.860001  16070.90042  108.66586  160.69112  105.756086   
BSC     6.574024   72.356289     0.443774   0.246291  -0.020545     0.21098   

                Gamma  
MC_EU        0.013229  
MC_AM  1617605.941605  
BSC          0.016061  
Put: 
           Price      Strike     Delta      Vega     Theta       Rho  \
MC_EU   4.669879   52.356289 -0.248876  0.200663 -0.014491 -0.202521   
MC_AM  32.767698  232.860001  0.153907   0.63779 -0.045441 -4.485097   
BSC      4.50634   52.356289 -0.248589  0.197558 -0.014252 -0.200074   

            Gamma  
MC_EU    0.028775  
MC_AM  161.655364  
BSC      0.012883  


Monte Carlo 95% CI error:  0.5179643595308595


In [ ]:


# Comparison for a position with and without delta hedge
K = arkq_stock.loc[arkq_stock.index[0], 'Close'] + 20
tao = 1
sigma = vol_hist

portfolio = HedgedPortfolio(arkq_stock[0:]['Close'], K=K, tao=tao, sigma=sigma)


In [ ]:
print(portfolio.df)
print('Risk free rate: ', portfolio.r)
print('PnL unhedged: ', portfolio.df.pnl_unhedged.sum())
print('PnL hedged:   ', portfolio.df.pnl.sum())